## 2570 Proj 3
___Warren Shi___

In [1]:
## Set up 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

%matplotlib inline

### Q1 

### a). simulation function

In [2]:
## i) function for simulation
'''
Function sim_wt
This function takes the intensity parameter alpha and number of simulations n to simulate n iid wait times for 
a Poisson Process with intensity alpha. 

parameters:
    alpha: double, the intensity parameter
    n    : double, number of simulations 
    
output:
    np array of length n, the n simulated wait times
'''
def sim_wt(alpha, n):
    return -np.log(1 - np.random.uniform(0,1,n)) / alpha

In [30]:
## ii) find N
0.01/st.norm.ppf(0.995)/np.sqrt(150)
N = 1000

In [31]:
## iii) find mean
np.mean(sim_wt(3, N))

0.33852983725652586